##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TF2 ワークフローで TF1.x モデルを使用する


<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/guide/migrate/model_mapping"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org で表示</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/guide/migrate/model_mapping.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Colab で実行</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/guide/migrate/model_mapping.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub で表示</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/guide/migrate/model_mapping.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a>
</td>
</table>

このガイドでは、モデリングコードへの変更を最小限に抑えて、Eager execution、`tf.function`、分散ストラテジーなどの TF2 ワークフローで既存の TF1.x モデルを使用するために使用できる[モデリングコード Shim](https://en.wikipedia.org/wiki/Shim_(computing)) の概要と例を示します。

## 利用範囲

このガイドで説明されている Shim は、以下に依存する TF1.x モデルのために設計されています。

1. 変数の作成と再利用を制御する `tf.compat.v1.get_variable` と `tf.compat.v1.variable_scope`、および
2. 重みと正則化損失を追跡するための `tf.compat.v1.global_variables()`、`tf.compat.v1.trainable_variables`、`tf.compat.v1.lossesget_regularization_losses()`、および `tf.compat.v1.get_collection()` などのグラフコレクションベースの API。

`tf.compat.v1.layer`、`tf.contrib.layers` API、および [TensorFlow-Slim](https://github.com/google-research/tf-slim) の上に構築されたほとんどのモデルで使用できます。

Shim は、次の TF1.x モデルでは必要**ありません**。

1. `model.trainable_weights` と `model.losses` を介して、すべてのトレーニング可能な重みと正則化損失を既に追跡しているスタンドアロンの Keras モデル。
2. `module.trainable_variables` を介してトレーニング可能なすべての重みを既に追跡し、まだ作成されていない場合にのみ重みを作成する `tf.Module`。

これらのモデルは、Eager execution と `tf.function` を使用して TF2 ですぐに使える可能性があります。

## セットアップ

TensorFlow と他の依存関係をインポートします。

In [ ]:
!pip uninstall -y -q tensorflow

In [ ]:
# Install tf-nightly as the DeterministicRandomTestTool is available only in
# Tensorflow 2.8

!pip install -q tf-nightly

In [ ]:
import tensorflow as tf
import tensorflow.compat.v1 as v1
import sys
import numpy as np

from contextlib import contextmanager

## `track_tf1_style_variables` デコレータ

このガイドで説明されている重要な Shim は `tf.keras.layers.Layer` に属するメソッド内で使用できるデコレータ `tf.compat.v1.keras.utils.track_tf1_style_variables` とTF1.x スタイルの重みを追跡し、正則化の損失を捕捉する `tf.Module` です。

`tf.keras.layers.Layer` または `tf.Module` の呼び出しメソッドを `tf.compat.v1.keras.utils.track_tf1_style_variables` でデコレートすると、`tf.compat.v1.get_variable`（および拡張 `tf.compat.v1.layers`）を介して変数の作成と再利用がデコレートされたメソッド内で正しく機能します。呼び出しごとに常に新しい変数を作成する必要はありません。また、レイヤーまたはモジュールは、デコレートされたメソッド内で `get_variable` を介して作成またはアクセスされた重みを暗黙的に追跡します。

標準の `layer.variable`/`module.variable` などで重み自体を追跡することに加えて、 メソッドが `tf.keras.layers.Layer` に属する場合、`get_variable` または `tf.compat.v1.layers` 正則化引数は、標準の `layer.losses` プロパティの下でレイヤーによって追跡されます。

この追跡メカニズムにより、TF2 の動作が有効になっている場合でも、Keras レイヤーまたは TF2 の `tf.Module` 内で TF1.x スタイルのモデルフォワードパスコードの大規模なクラスを使用できます。


## 使用例

以下の使用例は、`tf.keras.layers.Layer` メソッドをデコレートするために使用されるモデリング Shim を示していますが、Keras 機能と特に相互作用する場合を除き `tf.Module` をデコレートするときも適用できます。

### tf.compat.v1.get_variable で構築されたレイヤー

以下のように、`tf.compat.v1.get_variable` の上に直接実装されたレイヤーがあるとします。

```python
def dense(self, inputs, units):
  out = inputs
  with tf.compat.v1.variable_scope("dense"):
    # The weights are created with a `regularizer`,
    kernel = tf.compat.v1.get_variable(
        shape=[out.shape[-1], units],
        regularizer=tf.keras.regularizers.L2(),
        initializer=tf.compat.v1.initializers.glorot_normal,
        name="kernel")
    bias = tf.compat.v1.get_variable(
        shape=[units,],
        initializer=tf.compat.v1.initializers.zeros,
        name="bias")
    out = tf.linalg.matmul(out, kernel)
    out = tf.compat.v1.nn.bias_add(out, bias)
  return out
```

Shim を使用してレイヤーに変換し、入力で呼び出します。

In [ ]:
class DenseLayer(tf.keras.layers.Layer):

  def __init__(self, units, *args, **kwargs):
    super().__init__(*args, **kwargs)
    self.units = units

  @tf.compat.v1.keras.utils.track_tf1_style_variables
  def call(self, inputs):
    out = inputs
    with tf.compat.v1.variable_scope("dense"):
      # The weights are created with a `regularizer`,
      # so the layer should track their regularization losses
      kernel = tf.compat.v1.get_variable(
          shape=[out.shape[-1], self.units],
          regularizer=tf.keras.regularizers.L2(),
          initializer=tf.compat.v1.initializers.glorot_normal,
          name="kernel")
      bias = tf.compat.v1.get_variable(
          shape=[self.units,],
          initializer=tf.compat.v1.initializers.zeros,
          name="bias")
      out = tf.linalg.matmul(out, kernel)
      out = tf.compat.v1.nn.bias_add(out, bias)
    return out

layer = DenseLayer(10)
x = tf.random.normal(shape=(8, 20))
layer(x)

標準の Keras レイヤーのように、追跡された変数とキャプチャされた正則化損失にアクセスします。

In [ ]:
layer.trainable_variables
layer.losses

レイヤーを呼び出すたびに重みが再利用されることを確認するには、すべての重みをゼロに設定して、レイヤーを再度呼び出します。

In [ ]:
print("Resetting variables to zero:", [var.name for var in layer.trainable_variables])

for var in layer.trainable_variables:
  var.assign(var * 0.0)

# Note: layer.losses is not a live view and
# will get reset only at each layer call
print("layer.losses:", layer.losses)
print("calling layer again.")
out = layer(x)
print("layer.losses: ", layer.losses)
out

変換されたレイヤーは、Keras 機能モデルの構築でも直接使用できます。

In [ ]:
inputs = tf.keras.Input(shape=(20))
outputs = DenseLayer(10)(inputs)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

x = tf.random.normal(shape=(8, 20))
model(x)

# Access the model variables and regularization losses
model.weights
model.losses

### `tf.compat.v1.layers` で構築されたモデル

以下のように、レイヤーまたはモデルが `tf.compat.v1.layers` の上に直接実装されているとします。

```python
def model(self, inputs, units):
  with tf.compat.v1.variable_scope('model'):
    out = tf.compat.v1.layers.conv2d(
        inputs, 3, 3,
        kernel_regularizer="l2")
    out = tf.compat.v1.layers.flatten(out)
    out = tf.compat.v1.layers.dense(
        out, units,
        kernel_regularizer="l2")
    return out
```

Shim を使用してレイヤーに変換し、入力で呼び出します。

In [ ]:
class CompatV1LayerModel(tf.keras.layers.Layer):

  def __init__(self, units, *args, **kwargs):
    super().__init__(*args, **kwargs)
    self.units = units

  @tf.compat.v1.keras.utils.track_tf1_style_variables
  def call(self, inputs):
    with tf.compat.v1.variable_scope('model'):
      out = tf.compat.v1.layers.conv2d(
          inputs, 3, 3,
          kernel_regularizer="l2")
      out = tf.compat.v1.layers.flatten(out)
      out = tf.compat.v1.layers.dense(
          out, self.units,
          kernel_regularizer="l2")
      return out

layer = CompatV1LayerModel(10)
x = tf.random.normal(shape=(8, 5, 5, 5))
layer(x)

警告: 安全上の理由から、空でない文字列 `variable_scope` 内にすべての `tf.compat.v1.layers` を配置してください。これは、自動生成された名前を持つ `tf.compat.v1.layers` が、変数スコープの外で常に名前を自動インクリメントするためです。これは、レイヤー/モジュールを呼び出すたびに、要求された変数名が一致しないことを意味します。したがって、既に作成された重みを再利用するのではなく、呼び出しごとに新しい変数のセットを作成します。

標準の Keras レイヤーのように、追跡された変数とキャプチャされた正則化損失にアクセスします。

In [ ]:
layer.trainable_variables
layer.losses

レイヤーを呼び出すたびに重みが再利用されることを確認するには、すべての重みをゼロに設定して、レイヤーを再度呼び出します。

In [ ]:
print("Resetting variables to zero:", [var.name for var in layer.trainable_variables])

for var in layer.trainable_variables:
  var.assign(var * 0.0)

out = layer(x)
print("layer.losses: ", layer.losses)
out

変換されたレイヤーは、Keras 機能モデルの構築でも直接使用できます。

In [ ]:
inputs = tf.keras.Input(shape=(5, 5, 5))
outputs = CompatV1LayerModel(10)(inputs)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

x = tf.random.normal(shape=(8, 5, 5, 5))
model(x)

In [ ]:
# Access the model variables and regularization losses
model.weights
model.losses

### バッチ正則化の更新とモデルの `training` 引数をキャプチャする

TF1.x では、次のようにバッチ正則化を実行します。

```python
  x_norm = tf.compat.v1.layers.batch_normalization(x, training=training)

  # ...

  update_ops = tf.compat.v1.get_collection(tf.GraphKeys.UPDATE_OPS)
  train_op = optimizer.minimize(loss)
  train_op = tf.group([train_op, update_ops])
```

注意点:

1. バッチ正則化移動平均の更新は、レイヤーとは別に呼び出された `get_collection` によって追跡されます
2. `tf.compat.v1.layers.batch_normalization` には `training` 引数が必要です（通常、TF-Slim バッチ正則化レイヤーを使用する場合は `is_training` と呼ばれます）

TF2 では、[Eager execution](https://www.tensorflow.org/guide/eager) と自動制御の依存関係により、バッチ正則化移動平均更新がすぐに実行されます。これらを更新コレクションから個別に収集し、明示的な制御の依存関係として追加する必要はありません。

さらに、`tf.keras.layers.Layer` のフォワードパスメソッドに `training` 引数を与えると、Keras はその時点のトレーニングフェーズとネストされたレイヤーを他のレイヤーと同じように渡すことができます。Keras が `training` 引数を処理する方法の詳細については、`tf.keras.Model` の API ドキュメントを参照してください。

`tf.Module` メソッドをデコレートする場合は、必要に応じてすべての `training` 引数を手動で渡す必要があります。ただし、バッチ正則化移動平均更新は、明示的な制御依存関係を必要とせずに自動的に適用されます。

次のコードスニペットは、Shim にバッチ正則化レイヤーを埋め込む方法と、それを Keras モデルで使用する方法を示しています （`tf.keras.layers.Layer` に適用可能）。

In [ ]:
class CompatV1BatchNorm(tf.keras.layers.Layer):

  @tf.compat.v1.keras.utils.track_tf1_style_variables
  def call(self, inputs, training=None):
    print("Forward pass called with `training` =", training)
    with v1.variable_scope('batch_norm_layer'):
      return v1.layers.batch_normalization(x, training=training)

In [ ]:
print("Constructing model")
inputs = tf.keras.Input(shape=(5, 5, 5))
outputs = CompatV1BatchNorm()(inputs)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

print("Calling model in inference mode")
x = tf.random.normal(shape=(8, 5, 5, 5))
model(x, training=False)

print("Moving average variables before training: ",
      {var.name: var.read_value() for var in model.non_trainable_variables})

# Notice that when running TF2 and eager execution, the batchnorm layer directly
# updates the moving averages while training without needing any extra control
# dependencies
print("calling model in training mode")
model(x, training=True)

print("Moving average variables after training: ",
      {var.name: var.read_value() for var in model.non_trainable_variables})


### 変数スコープに基づく変数の再利用

`get_variable` に基づくフォワードパスでの変数の作成は、TF1.x の変数スコープと同じ変数の命名と再利用のセマンティクスを維持します。上記のように、自動生成された名前を持つ任意の `tf.compat.v1.layers` に対して少なくとも 1 つの空でない外部スコープがある必要があります。

注意: 命名と再利用は、単一のレイヤー/モジュールインスタンス内に限定されます。1 つの Shim でデコレートされたレイヤーまたはモジュール内の `get_variable` への呼び出しは、レイヤーまたはモジュール内で作成された変数を参照できません。`get_variable` を介して変数にアクセスするのではなく、必要に応じて他の変数への Python 参照を直接使用することで、これを回避できます。

### Eager execution と `tf.function`

上記のように、`tf.keras.layers.Layer` と `tf.Module` のデコレートされたメソッドは、Eager execution の内部で実行され、`tf.function` とも互換性があります。これは、[pdb](https://docs.python.org/3/library/pdb.html) やその他の対話型ツールを使用して、実行中のフォワードパスをステップ実行できることを意味します

警告: `tf.function` の *内*から Shim でデコレートされたレイヤー/モジュール メソッドを呼び出すことは完全に安全ですが、これらの `tf.function` に `get_variable` 呼び出しが含まれている場合、Shim でデコレートされたメソッド内に `tf.function` を配置することは安全ではありません。`tf.function` を入力すると、`variable_scope` がリセットされ、Shim が模倣する TF1.x スタイルの変数スコープに基づく変数の再利用が、この設定で失敗します。

### 分散ストラテジー

`@track_tf1_style_variables` でデコレートされたレイヤーまたはモジュールメソッド内の `get_variable` への呼び出しは、内部で標準の `tf.Variable` 変数の作成を使用します。これは、`MirroredStrategy` や `TPUStrategy` など、`tf.distribute` で利用可能なさまざまな分散ストラテジーでそれらを使用できることを意味します。

## デコレートされた呼び出しで `tf.Variable`、`tf.Module`、`tf.keras.layers` および `tf.keras.models` をネストする

`tf.compat.v1.keras.utils.track_tf1_style_variables` でレイヤー呼び出しをデコレートすると、`tf.compat.v1.get_variable` を介して作成された（および再利用された）変数の自動暗黙的追跡のみが追加されます。典型的な Keras レイヤーやほとんどの `tf.Module` で使用される `tf.Variable` 呼び出しによって直接作成された重みはキャプチャしません。このセクションでは、これらのネストされたケースを処理する方法について説明します。


### （既存の使用法）`tf.keras.layers` および `tf.keras.models`

ネストされた Keras レイヤーとモデルの既存の使用法については、`tf.compat.v1.keras.utils.get_or_create_layer` を使用します。これは、既存の TF1.x にネストされた Keras の使用法の移行を容易にするためにのみ推奨されます。tf.Variable および tf.Module の新しいコードは、以下で説明するように明示的な属性設定を使用する必要があります。

`tf.compat.v1.keras.utils.get_or_create_layer` を使用するには、ネストされたモデルを構築するコードをメソッドにラップし、メソッドに渡します。以下に例を示します。

In [ ]:
class NestedModel(tf.keras.Model):

  def __init__(self, units, *args, **kwargs):
    super().__init__(*args, **kwargs)
    self.units = units

  def build_model(self):
    inp = tf.keras.Input(shape=(5, 5))
    dense_layer = tf.keras.layers.Dense(
        10, name="dense", kernel_regularizer="l2",
        kernel_initializer=tf.compat.v1.ones_initializer())
    model = tf.keras.Model(inputs=inp, outputs=dense_layer(inp))
    return model

  @tf.compat.v1.keras.utils.track_tf1_style_variables
  def call(self, inputs):
    # Get or create a nested model without assigning it as an explicit property
    model = tf.compat.v1.keras.utils.get_or_create_layer(
        "dense_model", self.build_model)
    return model(inputs)

layer = NestedModel(10)
layer(tf.ones(shape=(5,5)))

このメソッドにより、これらのネストされたレイヤーが正しく再利用され、TensorFlow によって追跡されることが保証されます。適切なメソッドでは `@track_tf1_style_variables` デコレータが引き続き必要であることに注意してください。`get_or_create_layer` に渡されるモデルビルダーメソッド（この場合は `self.build_model`）は、引数を取りません。

重みが追跡されます。

In [ ]:
assert len(layer.weights) == 2
weights = {x.name: x for x in layer.variables}

assert set(weights.keys()) == {"dense/bias:0", "dense/kernel:0"}

layer.weights

正則化損失も同様に追跡されます。

In [ ]:
tf.add_n(layer.losses)

### 増分移行: `tf.Variables` および `tf.Modules`

デコレートされたメソッドに `tf.Variable` 呼び出し、または `tf.Module` を埋め込む必要がある場合（たとえば、このガイドの後半で説明されている非レガシー TF2 API への段階的な移行に従っている場合など）、次の要件に従って、これらを明示的に追跡する必要があります。

- 変数/モジュール/レイヤーが一度だけ作成されることを明示的に確認する
- [典型的なモジュールまたはレイヤー](https://www.tensorflow.org/guide/intro_to_modules#defining_models_and_layers_in_tensorflow)を定義するときと同じように、それらをインスタンス属性として明示的に添付する
- 後続の呼び出しで、作成済みのオブジェクトを明示的に再利用する

これにより、重みが呼び出しごとに新しく作成されず、正しく再利用されることが保証されます。さらに、既存の重みと正則化の損失が追跡されることも保証されます。

以下に例を示します。

In [ ]:
class NestedLayer(tf.keras.layers.Layer):

  def __init__(self, units, *args, **kwargs):
    super().__init__(*args, **kwargs)
    self.units = units

  @tf.compat.v1.keras.utils.track_tf1_style_variables
  def __call__(self, inputs):
    out = inputs
    with tf.compat.v1.variable_scope("inner_dense"):
      # The weights are created with a `regularizer`,
      # so the layer should track their regularization losses
      kernel = tf.compat.v1.get_variable(
          shape=[out.shape[-1], self.units],
          regularizer=tf.keras.regularizers.L2(),
          initializer=tf.compat.v1.initializers.glorot_normal,
          name="kernel")
      bias = tf.compat.v1.get_variable(
          shape=[self.units,],
          initializer=tf.compat.v1.initializers.zeros,
          name="bias")
      out = tf.linalg.matmul(out, kernel)
      out = tf.compat.v1.nn.bias_add(out, bias)
    return out

class WrappedDenseLayer(tf.keras.layers.Layer):

  def __init__(self, units, **kwargs):
    super().__init__(**kwargs)
    self.units = units
    # Only create the nested tf.variable/module/layer/model
    # once, and then reuse it each time!
    self._dense_layer = NestedLayer(self.units)

  @tf.compat.v1.keras.utils.track_tf1_style_variables
  def call(self, inputs):
    with tf.compat.v1.variable_scope('outer'):
      outputs = tf.compat.v1.layers.dense(inputs, 3)
      outputs = tf.compat.v1.layers.dense(inputs, 4)
      return self._dense_layer(outputs)

layer = WrappedDenseLayer(10)

layer(tf.ones(shape=(5, 5)))

`track_tf1_style_variables` デコレータで装飾されている場合でも、ネストされたモジュールを明示的に追跡する必要があることに注意してください。デコレートされたメソッドを持つ各モジュール/レイヤーには、それに関連付けられた独自の変数ストアがあるためです。

重みは正しく追跡されます。

In [ ]:
assert len(layer.weights) == 6
weights = {x.name: x for x in layer.variables}

assert set(weights.keys()) == {"outer/inner_dense/bias:0",
                               "outer/inner_dense/kernel:0",
                               "outer/dense/bias:0",
                               "outer/dense/kernel:0",
                               "outer/dense_1/bias:0",
                               "outer/dense_1/kernel:0"}

layer.trainable_weights

正則化損失も同様に追跡されます。

In [ ]:
layer.losses

`NestedLayer` が Keras 以外の `tf.Module` である場合、変数は引き続き追跡されますが、正則化の損失は自動的に追跡されないため、別に明示的に追跡する必要があります。

### 変数名に関するガイダンス

明示的な `tf.Variable` 呼び出しと Keras レイヤーは、馴染まれている `get_variable` と `variable_scopes` の組み合わせとは異なるレイヤー名/変数名自動生成メカニズムを使用します。TF1.x グラフから TF2 の Eager execution と `tf.function` に移行する場合でも、Shim は、変数名を `get_variable` により作成された変数と一致させますが、`tf.Variable` 呼び出しのために生成された変数名と、メソッドデコレータ内に埋め込む Keras レイヤーでは同じことを保証することはできません。TF2 の Eager execution と `tf.function` では、複数の変数が同じ名前を共有することも可能です。

このガイドで後述する、正確性の検証と TF1.x チェックポイントのマッピングに関するセクションに従うときは、このことに特に注意する必要があります。

### デコレートされたメソッドで `tf.compat.v1.make_template` を使用する

**`tf.compat.v1.make_template` を使用する代わりに、TF2 の上の薄いレイヤーである`tf.compat.v1.keras.utils.track_tf1_style_variables` を直接使用することを強くお勧めします**。

すでに `tf.compat.v1.make_template` に依存する以前の TF1.x コードについては、このセクションのガイダンスに従ってください。

`tf.compat.v1.make_template` は `get_variable` を使用するコードをラップするため、`track_tf1_style_variables` デコレータを使用すると、レイヤー呼び出しでこれらのテンプレートを使用して、重みと正則化損失を正常に追跡できます。

ただし、必ず `make_template` を 1 回だけ呼び出してから、各レイヤー呼び出しで同じテンプレートを再利用してください。そうしないと、新しい変数セットとともにレイヤーを呼び出すたびに、新しいテンプレートが作成されます。

以下に例を示します。

In [ ]:
class CompatV1TemplateScaleByY(tf.keras.layers.Layer):

  def __init__(self, **kwargs):
    super().__init__(**kwargs)
    def my_op(x, scalar_name):
      var1 = tf.compat.v1.get_variable(scalar_name,
                            shape=[],
                            regularizer=tf.compat.v1.keras.regularizers.L2(),
                            initializer=tf.compat.v1.constant_initializer(1.5))
      return x * var1
    self.scale_by_y = tf.compat.v1.make_template('scale_by_y', my_op, scalar_name='y')

  @tf.compat.v1.keras.utils.track_tf1_style_variables
  def call(self, inputs):
    with tf.compat.v1.variable_scope('layer'):
      # Using a scope ensures the `scale_by_y` name will not be incremented
      # for each instantiation of the layer.
      return self.scale_by_y(inputs)

layer = CompatV1TemplateScaleByY()

out = layer(tf.ones(shape=(2, 3)))
print("weights:", layer.weights)
print("regularization loss:", layer.losses)
print("output:", out)

警告: `make_template` で作成された同じテンプレートを複数のレイヤーインスタンスで共有しないでください。Shim デコレータの変数および正則化損失追跡メカニズムが壊れる可能性があるためです。さらに、複数のレイヤーインスタンス内で同じ `make_template` 名を使用する場合は、作成したテンプレートの使用法を `variable_scope` 内にネストする必要があります。そうでない場合、テンプレートの `variable_scope` の生成された名前は、レイヤーの新しいインスタンスごとに増加します。これにより、重みの名前が予期しない方法で変更される可能性があります。

## ネイティブ TF2 への段階的な移行

前述のように、`track_tf1_style_variables` を使用すると、TF2 スタイルのオブジェクト指向の `tf.Variable`/`tf.keras.layers.Layer`/`tf.Module`  の使用法と同じデコレートされたモジュール/レイヤー内の従来の `tf.compat.v1.get_variable`/`tf.compat.v1.layers` スタイルの使用法を併用できます。

これは、TF1.x モデルを TF2 と完全に互換性のあるものにした後、すべての新しいモデルコンポーネントをネイティブ（非 `tf.compat.v1`）TF2 API で記述し、以前のコードと相互運用できることを意味します。

ただし、以前モデルコンポーネントを変更し続ける場合は、従来のスタイルの `tf.compat.v1` の使用法を新しく記述された TF2 コードに推奨される純粋にネイティブなオブジェクト指向 API に段階的に切り替えることもできます。

`tf.compat.v1.get_variable` の使用法は、Keras レイヤー/モデルをデコレートしている場合は `self.add_weight` 呼び出し、Keras オブジェクトまたは `tf.Module` をデコレートしている場合は `tf.Variable` 呼び出しに置き換えることがます。

関数型とオブジェクト指向の両方の `tf.compat.v1.layers` は、通常、引数を変更することなく、同等の `tf.keras.layers` レイヤーに置き換えることができます。

`track_tf1_style_variables` を使用する純粋なネイティブ API への段階的な移行中に、モデルの一部または共通パターンを個々のレイヤー/モジュールに含めることもできます。

### Slim と contrib.layers に関する注意

以前の TF 1.x コードの多くは、TF 1.x に `tf.contrib.layers` としてパッケージ化された [Slim](https://ai.googleblog.com/2016/08/tf-slim-high-level-library-to-define.html) ライブラリを使用しています。Slim を使用したコードをネイティブ TF 2 に変換することは、`v1.layers` を変換するよりも複雑です。そのため、まず Slim コードを `v1.layers` に変換してから、Keras に変換する方が理にかなっています。以下は、Slim コードを変換するための一般的なガイダンスです。

- すべての引数が明示的であることを確認します。可能であれば `arg_scopes` を削除します。使用する必要がある場合は、`normalizer_fn` と `activation_fn` をそれぞれのレイヤーに分割します。
- 分離可能な畳み込みレイヤーは 1 つまたはそれ以上の異なる Keras レイヤー（深さ、点、分離可能な Keras レイヤー）にマップします。
- Slim と `v1.layers` には異なる引数名とデフォルト値があります。
- 一部の引数には異なるスケールがあります。

### チェックポイントの互換性を無視したネイティブ TF2 への移行

次のコードサンプルは、チェックポイントの互換性を考慮せずにモデルを純粋なネイティブ API に段階的に移行する方法を示しています。

In [ ]:
class CompatModel(tf.keras.layers.Layer):

  def __init__(self, units, *args, **kwargs):
    super().__init__(*args, **kwargs)
    self.units = units

  @tf.compat.v1.keras.utils.track_tf1_style_variables
  def call(self, inputs, training=None):
    with tf.compat.v1.variable_scope('model'):
      out = tf.compat.v1.layers.conv2d(
          inputs, 3, 3,
          kernel_regularizer="l2")
      out = tf.compat.v1.layers.flatten(out)
      out = tf.compat.v1.layers.dropout(out, training=training)
      out = tf.compat.v1.layers.dense(
          out, self.units,
          kernel_regularizer="l2")
      return out


次に、`compat.v1` API をネイティブオブジェクト指向の同等のものに部分的に置き換えます。レイヤーコンストラクタで作成された Keras オブジェクトに畳み込みレイヤーを切り替えることから始めます。

In [ ]:
class PartiallyMigratedModel(tf.keras.layers.Layer):

  def __init__(self, units, *args, **kwargs):
    super().__init__(*args, **kwargs)
    self.units = units
    self.conv_layer = tf.keras.layers.Conv2D(
      3, 3,
      kernel_regularizer="l2")

  @tf.compat.v1.keras.utils.track_tf1_style_variables
  def call(self, inputs, training=None):
    with tf.compat.v1.variable_scope('model'):
      out = self.conv_layer(inputs)
      out = tf.compat.v1.layers.flatten(out)
      out = tf.compat.v1.layers.dropout(out, training=training)
      out = tf.compat.v1.layers.dense(
          out, self.units,
          kernel_regularizer="l2")
      return out


[`v1.keras.utils.DeterministicRandomTestTool`](https://www.tensorflow.org/api_docs/python/tf/compat/v1/keras/utils/DeterministicRandomTestTool) クラスを使用して、この段階的な変更によってモデルが以前と同じ動作をすることを確認します。

In [ ]:
random_tool = v1.keras.utils.DeterministicRandomTestTool(mode='num_random_ops')
with random_tool.scope():
  tf.keras.utils.set_random_seed(42)
  layer = CompatModel(10)

  inputs = tf.random.normal(shape=(10, 5, 5, 5))
  original_output = layer(inputs)

  # Grab the regularization loss as well
  original_regularization_loss = tf.math.add_n(layer.losses)

print(original_regularization_loss)

In [ ]:
random_tool = v1.keras.utils.DeterministicRandomTestTool(mode='num_random_ops')
with random_tool.scope():
  tf.keras.utils.set_random_seed(42)
  layer = PartiallyMigratedModel(10)

  inputs = tf.random.normal(shape=(10, 5, 5, 5))
  migrated_output = layer(inputs)

  # Grab the regularization loss as well
  migrated_regularization_loss = tf.math.add_n(layer.losses)

print(migrated_regularization_loss)

In [ ]:
# Verify that the regularization loss and output both match
np.testing.assert_allclose(original_regularization_loss.numpy(), migrated_regularization_loss.numpy())
np.testing.assert_allclose(original_output.numpy(), migrated_output.numpy())

個々の `compat.v1.layers` をすべてネイティブ Keras レイヤーに置き換えました。

In [ ]:
class NearlyFullyNativeModel(tf.keras.layers.Layer):

  def __init__(self, units, *args, **kwargs):
    super().__init__(*args, **kwargs)
    self.units = units
    self.conv_layer = tf.keras.layers.Conv2D(
      3, 3,
      kernel_regularizer="l2")
    self.flatten_layer = tf.keras.layers.Flatten()
    self.dense_layer = tf.keras.layers.Dense(
      self.units,
      kernel_regularizer="l2")

  @tf.compat.v1.keras.utils.track_tf1_style_variables
  def call(self, inputs):
    with tf.compat.v1.variable_scope('model'):
      out = self.conv_layer(inputs)
      out = self.flatten_layer(out)
      out = self.dense_layer(out)
      return out


In [ ]:
random_tool = v1.keras.utils.DeterministicRandomTestTool(mode='num_random_ops')
with random_tool.scope():
  tf.keras.utils.set_random_seed(42)
  layer = NearlyFullyNativeModel(10)

  inputs = tf.random.normal(shape=(10, 5, 5, 5))
  migrated_output = layer(inputs)

  # Grab the regularization loss as well
  migrated_regularization_loss = tf.math.add_n(layer.losses)

print(migrated_regularization_loss)

In [ ]:
# Verify that the regularization loss and output both match
np.testing.assert_allclose(original_regularization_loss.numpy(), migrated_regularization_loss.numpy())
np.testing.assert_allclose(original_output.numpy(), migrated_output.numpy())

最後に、残りの（不要になった）`variable_scope` の使用と `track_tf1_style_variables` デコレータ自体を削除します。

完全にネイティブ API を使用するモデルのバージョンになりました。

In [ ]:
class FullyNativeModel(tf.keras.layers.Layer):

  def __init__(self, units, *args, **kwargs):
    super().__init__(*args, **kwargs)
    self.units = units
    self.conv_layer = tf.keras.layers.Conv2D(
      3, 3,
      kernel_regularizer="l2")
    self.flatten_layer = tf.keras.layers.Flatten()
    self.dense_layer = tf.keras.layers.Dense(
      self.units,
      kernel_regularizer="l2")

  def call(self, inputs):
    out = self.conv_layer(inputs)
    out = self.flatten_layer(out)
    out = self.dense_layer(out)
    return out


In [ ]:
random_tool = v1.keras.utils.DeterministicRandomTestTool(mode='num_random_ops')
with random_tool.scope():
  tf.keras.utils.set_random_seed(42)
  layer = FullyNativeModel(10)

  inputs = tf.random.normal(shape=(10, 5, 5, 5))
  migrated_output = layer(inputs)

  # Grab the regularization loss as well
  migrated_regularization_loss = tf.math.add_n(layer.losses)

print(migrated_regularization_loss)

In [ ]:
# Verify that the regularization loss and output both match
np.testing.assert_allclose(original_regularization_loss.numpy(), migrated_regularization_loss.numpy())
np.testing.assert_allclose(original_output.numpy(), migrated_output.numpy())

### ネイティブ TF2 への移行中にチェックポイントの互換性を維持する

上記のネイティブ TF2 API への移行プロセスでは、変数名（Keras API は非常に異なる重み名を生成するため）と、モデル内の異なる重みを指すオブジェクト指向パスの両方が変更されました。これらの変更により、既存の TF1 スタイルの名前ベースのチェックポイントと TF2 スタイルのオブジェクト指向チェックポイントの両方が機能しなくなります。

ただし、場合によっては、元の名前ベースのチェックポイントを使用して、[TF1.x チェックポイントの再利用ガイド](./migrating_checkpoints.ipynb)で詳しく説明されているようなアプローチを使用して、新しい名前への変数のマッピングを見つけられる場合があります。

以下にヒントを示します。

- 変数はすべて設定が可能な `name` 引数を持ちます。
- また、Keras モデルは `name` 引数を取り、それらの変数のためのプレフィックスとして設定されます。
- `v1.name_scope` 関数は、変数名のプレフィックスの設定に使用できます。これは `tf.variable_scope` とは大きく異なります。これは名前だけに影響するもので、変数と再利用の追跡はしません。

上記を念頭に置いて、次のコードサンプルは、チェックポイントを同時に更新しながら、モデルの一部を段階的に更新するためにコードに適応できるワークフローを示しています。

注意: Keras レイヤーでの変数の命名は複雑であるため、これがすべてのユースケースで機能するとは限りません。

1. まず、関数型の `tf.compat.v1.layers` をオブジェクト指向のバージョンに切り替えます。

In [ ]:
class FunctionalStyleCompatModel(tf.keras.layers.Layer):

  @tf.compat.v1.keras.utils.track_tf1_style_variables
  def call(self, inputs, training=None):
    with tf.compat.v1.variable_scope('model'):
      out = tf.compat.v1.layers.conv2d(
          inputs, 3, 3,
          kernel_regularizer="l2")
      out = tf.compat.v1.layers.conv2d(
          out, 4, 4,
          kernel_regularizer="l2")
      out = tf.compat.v1.layers.conv2d(
          out, 5, 5,
          kernel_regularizer="l2")
      return out

layer = FunctionalStyleCompatModel()
layer(tf.ones(shape=(10, 10, 10, 10)))
[v.name for v in layer.weights]

1. 次に、compat.v1.layer オブジェクトと `compat.v1.get_variable` によって作成された変数をメソッドが `track_tf1_style_variables` でデコレートされた `tf.keras.layers.Layer`/`tf.Module` オブジェクトのプロパティとして割り当てます。（オブジェクト指向の TF2 スタイルのチェックポイントは、変数名によるパスと新しいオブジェクト指向のパスの両方を保存することに注意してください）。

In [ ]:
class OOStyleCompatModel(tf.keras.layers.Layer):

  def __init__(self, *args, **kwargs):
    super().__init__(*args, **kwargs)
    self.conv_1 = tf.compat.v1.layers.Conv2D(
          3, 3,
          kernel_regularizer="l2")
    self.conv_2 = tf.compat.v1.layers.Conv2D(
          4, 4,
          kernel_regularizer="l2")

  @tf.compat.v1.keras.utils.track_tf1_style_variables
  def call(self, inputs, training=None):
    with tf.compat.v1.variable_scope('model'):
      out = self.conv_1(inputs)
      out = self.conv_2(out)
      out = tf.compat.v1.layers.conv2d(
          out, 5, 5,
          kernel_regularizer="l2")
      return out

layer = OOStyleCompatModel()
layer(tf.ones(shape=(10, 10, 10, 10)))
[v.name for v in layer.weights]

1. この時点で読み込まれたチェックポイントを再保存して、変数名（compat.v1.layers の場合）またはオブジェクト指向オブジェクトグラフの両方でパスを保存します。

In [ ]:
weights = {v.name: v for v in layer.weights}
assert weights['model/conv2d/kernel:0'] is layer.conv_1.kernel
assert weights['model/conv2d_1/bias:0'] is layer.conv_2.bias

1. オブジェクト指向の `compat.v1.layers` をネイティブ Keras レイヤーに置き換えながら、最近保存したチェックポイントを読み込めるようになりました。置き換えられたレイヤーの自動生成された `variable_scopes` を引き続き記録することにより、残りの `compat.v1.layers` の変数名を確実に保持します。これらの置き換えられたレイヤー/変数は、変数名パスの代わりに、チェックポイント内の変数へのオブジェクト属性パスのみを使用するようになりました。

一般に、プロパティにアタッチされた変数での `compat.v1.get_variable` の使用を次のように置き換えることができます。

- `tf.Variable` を使用するように置き換えます。**または**
- [`tf.keras.layers.Layer.add_weight`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Layer#add_weight) を使用してそれらを更新します。一度にすべてのレイヤーを切り替えない場合、`name` 引数がない残りの `compat.v1.layers` の自動生成されたレイヤー/変数の命名が変更される可能性があることに注意してください。その場合、削除された `compat.v1.layer` の生成されたスコープ名に対応する `variable_scope` を手動で開閉し、残りの `compat.v1.layers` の変数名を同じにしておく必要があります。そうしないと、既存のチェックポイントからのパスが競合する可能性があり、チェックポイントの読み込みが正しく動作しなくなります。


In [ ]:
def record_scope(scope_name):
  """Record a variable_scope to make sure future ones get incremented."""
  with tf.compat.v1.variable_scope(scope_name):
    pass

class PartiallyNativeKerasLayersModel(tf.keras.layers.Layer):

  def __init__(self, *args, **kwargs):
    super().__init__(*args, **kwargs)
    self.conv_1 = tf.keras.layers.Conv2D(
          3, 3,
          kernel_regularizer="l2")
    self.conv_2 = tf.keras.layers.Conv2D(
          4, 4,
          kernel_regularizer="l2")

  @tf.compat.v1.keras.utils.track_tf1_style_variables
  def call(self, inputs, training=None):
    with tf.compat.v1.variable_scope('model'):
      out = self.conv_1(inputs)
      record_scope('conv2d') # Only needed if follow-on compat.v1.layers do not pass a `name` arg
      out = self.conv_2(out)
      record_scope('conv2d_1') # Only needed if follow-on compat.v1.layers do not pass a `name` arg
      out = tf.compat.v1.layers.conv2d(
          out, 5, 5,
          kernel_regularizer="l2")
      return out

layer = PartiallyNativeKerasLayersModel()
layer(tf.ones(shape=(10, 10, 10, 10)))
[v.name for v in layer.weights]

変数を構築した後、このステップでチェックポイントを保存すると、最新の利用可能なオブジェクトパス***のみ***が含まれるようになります。

削除された `compat.v1.layers` のスコープを記録して、残りの `compat.v1.layers` の自動生成された重み名を保持します。

In [ ]:
weights = set(v.name for v in layer.weights)
assert 'model/conv2d_2/kernel:0' in weights
assert 'model/conv2d_2/bias:0' in weights

1. モデル内のすべての `compat.v1.layers` と `compat.v1.get_variable` を完全にネイティブな同等のものに置き換えるまで、上記の手順を繰り返します。

In [ ]:
class FullyNativeKerasLayersModel(tf.keras.layers.Layer):

  def __init__(self, *args, **kwargs):
    super().__init__(*args, **kwargs)
    self.conv_1 = tf.keras.layers.Conv2D(
          3, 3,
          kernel_regularizer="l2")
    self.conv_2 = tf.keras.layers.Conv2D(
          4, 4,
          kernel_regularizer="l2")
    self.conv_3 = tf.keras.layers.Conv2D(
          5, 5,
          kernel_regularizer="l2")


  def call(self, inputs, training=None):
    with tf.compat.v1.variable_scope('model'):
      out = self.conv_1(inputs)
      out = self.conv_2(out)
      out = self.conv_3(out)
      return out

layer = FullyNativeKerasLayersModel()
layer(tf.ones(shape=(10, 10, 10, 10)))
[v.name for v in layer.weights]

新しく更新されたチェックポイントが期待どおりに動作することを確認するためにテストすることを忘れないでください。このプロセスの段階的なステップごとに、[数値の正確性を検証するガイド](./validate_correctness.ipynb)に記載されている手法を適用して、移行したコードが正しく実行されるようにします。

## モデリング Shim で処理されない TF1.x から TF2 への動作変更

このガイドで説明されているモデリング Shim を使用することにより、Eager execution と `tf.function` を使用する際にコレクションに依存することなく、`get_variable`、`tf.compat.v1.layers`、および `variable_scope` で作成された変数、レイヤー、正則化の損失のセマンティクスが以前と同様に機能できるようになります。

これは、モデルのフォワードパスが依存している可能性がある***すべての*** TF1.x 固有のセマンティクスを網羅しているわけではありません。場合によっては、モデルのフォワードパスを TF2 で単独で実行するには、Shim が不十分な場合があります。TF1.x と TF2 の動作の違いについて詳しくは、[TF1.x と TF2 の動作ガイド](./tf1_vs_tf2)を参照してください。